In [10]:
import pytesseract
from PIL import Image, ImageFilter, ImageEnhance
import numpy as np
from skimage import io
from skimage.color import rgb2gray
from skimage.transform import rotate
from deskew import determine_skew
import glob
import cv2
import numpy as np  
import os
from tqdm.notebook import tqdm 
import time

path_to_tesseract = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [27]:
input_list_names = glob.glob("wszystkie_zdj/test/*",)
for photo_name in tqdm(input_list_names):
    name = photo_name.split("\\")[-1]
    name = name.split(".")[0]

    try:    
        image = io.imread(photo_name)
    except:
        print(f"Nie udało się otworzyć pliku {name}")
        continue
    angle = determine_skew(image)
    rotated = rotate(image, angle, resize=True) * 255

    if abs (angle) >= 30:
        io.imsave(f'deskewed/test/moreeq30/{name}.png', image.astype(np.uint8))
    else:
        io.imsave(f'deskewed/test/less30/{name}.png', rotated.astype(np.uint8))

  0%|          | 0/3492 [00:00<?, ?it/s]

C:\Users\adamm\AppData\Local\Temp\ipykernel_20988\1891972723.py:17: UserWarning: deskewed/test/less30/09473453-6a5e-40f6-a0ed-7b9d14658250.png is a low contrast image
  io.imsave(f'deskewed/test/less30/{name}.png', rotated.astype(np.uint8))
C:\Users\adamm\AppData\Local\Temp\ipykernel_20988\1891972723.py:17: UserWarning: deskewed/test/less30/0b416cca-bbd3-44f3-a29c-5192c6d795b0.png is a low contrast image
  io.imsave(f'deskewed/test/less30/{name}.png', rotated.astype(np.uint8))
C:\Users\adamm\AppData\Local\Temp\ipykernel_20988\1891972723.py:17: UserWarning: deskewed/test/less30/0cc8384c-7440-4499-83a4-33dc49b9b21d.png is a low contrast image
  io.imsave(f'deskewed/test/less30/{name}.png', rotated.astype(np.uint8))
C:\Users\adamm\AppData\Local\Temp\ipykernel_20988\1891972723.py:17: UserWarning: deskewed/test/less30/0f61d8da-4ebf-41b5-8409-3cbcb84a719d.png is a low contrast image
  io.imsave(f'deskewed/test/less30/{name}.png', rotated.astype(np.uint8))
C:\Users\adamm\AppData\Local\Temp\ip

Nie udało się otworzyć pliku de5e15cc-6a3b-4659-bb5e-2456ef87ea05


C:\Users\adamm\AppData\Local\Temp\ipykernel_20988\1891972723.py:17: UserWarning: deskewed/test/less30/e176e815-0848-49ee-ae3e-226feb9f2b67.png is a low contrast image
  io.imsave(f'deskewed/test/less30/{name}.png', rotated.astype(np.uint8))
C:\Users\adamm\AppData\Local\Temp\ipykernel_20988\1891972723.py:17: UserWarning: deskewed/test/less30/e6ec0e86-9254-422d-bdbb-e696dd7ec153.png is a low contrast image
  io.imsave(f'deskewed/test/less30/{name}.png', rotated.astype(np.uint8))
C:\Users\adamm\AppData\Local\Temp\ipykernel_20988\1891972723.py:17: UserWarning: deskewed/test/less30/ed9640d4-d410-478b-89c9-f67fa53a301c.png is a low contrast image
  io.imsave(f'deskewed/test/less30/{name}.png', rotated.astype(np.uint8))
C:\Users\adamm\AppData\Local\Temp\ipykernel_20988\1891972723.py:17: UserWarning: deskewed/test/less30/f3b21f34-e63d-4e3f-a152-e37eec4bb5c2.png is a low contrast image
  io.imsave(f'deskewed/test/less30/{name}.png', rotated.astype(np.uint8))
C:\Users\adamm\AppData\Local\Temp\ip

In [30]:
# increase contrast
input_list_names = glob.glob("deskewed/test/less30/*",)
for photo_name in tqdm(input_list_names):
    name = photo_name.split("\\")[-1]
    name = name.split(".")[0]
    original_image = io.imread(photo_name)
    image = Image.fromarray(original_image)
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1.5)
    image = np.array(image)
    io.imsave(f'contrast/test/less30/{name}.png', image.astype(np.uint8))


  0%|          | 0/3456 [00:00<?, ?it/s]

C:\Users\adamm\AppData\Local\Temp\ipykernel_20988\1174088497.py:11: UserWarning: contrast/test/less30/09473453-6a5e-40f6-a0ed-7b9d14658250.png is a low contrast image
  io.imsave(f'contrast/test/less30/{name}.png', image.astype(np.uint8))
C:\Users\adamm\AppData\Local\Temp\ipykernel_20988\1174088497.py:11: UserWarning: contrast/test/less30/0b416cca-bbd3-44f3-a29c-5192c6d795b0.png is a low contrast image
  io.imsave(f'contrast/test/less30/{name}.png', image.astype(np.uint8))
C:\Users\adamm\AppData\Local\Temp\ipykernel_20988\1174088497.py:11: UserWarning: contrast/test/less30/0cc8384c-7440-4499-83a4-33dc49b9b21d.png is a low contrast image
  io.imsave(f'contrast/test/less30/{name}.png', image.astype(np.uint8))
C:\Users\adamm\AppData\Local\Temp\ipykernel_20988\1174088497.py:11: UserWarning: contrast/test/less30/0fbb13ad-1688-4a8c-9861-86d998dfb546.png is a low contrast image
  io.imsave(f'contrast/test/less30/{name}.png', image.astype(np.uint8))
C:\Users\adamm\AppData\Local\Temp\ipykernel_

In [31]:
sharpen_filter = np.array([[-1, -1, -1],
                           [-1, 9, -1],
                           [-1, -1, -1]])

input_list_names = glob.glob("contrast/test/less30/*",)
for photo_name in tqdm(input_list_names):
    name = photo_name.split("\\")[-1]
    name = name.split(".")[0]
    original_image = io.imread(photo_name)
    sharp_image = cv2.filter2D(original_image, -1, sharpen_filter)
    sharp_image = Image.fromarray(sharp_image)
    sharp_image.save(f'sharpened/test/less30/{name}.png')


  0%|          | 0/3456 [00:00<?, ?it/s]

In [18]:
# iterate over the list of image paths
input_list_names = glob.glob("sharpened/test/less30/*",)
custom_config = r'--oem 3 --psm 6 -l pol'
photos_dict_less30 = {}
for photo_name in tqdm(input_list_names):
    name = photo_name.split("\\")[-1]
    name = name.split(".")[0]
    image = io.imread(photo_name)


    # Perform OCR using pytesseract
    text = pytesseract.image_to_string(image, config=custom_config)
    # count number of \n
    count = text.count("\n")
    # replace \n with space
    text = text.replace("\n", " ")
    # replace \x0c with space
    text = text.replace("\x0c", " ")
   

    photos_dict_less30[name] = (text, count)
    

  0%|          | 0/3456 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [9]:
# save dictionary to pickle
import pickle
with open('photos_dict_less30_test.pickle', 'wb') as handle:
    pickle.dump(photos_dict_less30, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [10]:
# iterate over the list of image paths
input_list_names = glob.glob("sharpened/test/moreeq30/*",)
custom_config = r'--oem 3 --psm 6 -l pol'
photos_dict_moreeq30 = {}
for photo_name in tqdm(input_list_names):
    name = photo_name.split("\\")[-1]
    name = name.split(".")[0]
    image = io.imread(photo_name)


    # Perform OCR using pytesseract
    text = pytesseract.image_to_string(image, config=custom_config)
    photos_dict_moreeq30[name] = text
    

  0%|          | 0/108 [00:00<?, ?it/s]

In [ ]:
# save dictionary to pickle
import pickle
with open('photos_dict_moreeq30_test.pickle', 'wb') as handle:
    pickle.dump(photos_dict_moreeq30, handle, protocol=pickle.HIGHEST_PROTOCOL)
    